In [1]:
import os
import glob
import csv

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

full_data_rows_list = []

for file in file_path_list:
    with open(file, 'r', encoding = 'utf8', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)

        for line in csvreader:
            full_data_rows_list.append(line)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as file:
    writer = csv.writer(file, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))



ModuleNotFoundError: No module named 'cassandra'

In [ ]:
from cassandra.cluster import Cluster

import csv

file = 'event_datafile_new.csv'
cluster = Cluster()

session = cluster.connect()

try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
        print(e)

try:
    session.set_keyspace('sparkify')
except Exception as e:
        print(e)

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = "CREATE TABLE IF NOT EXISTS song_length "
query = query + "(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_length (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userid INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT, PRIMARY KEY ((userid, sessionid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userid, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS users_song "
query = query + "(song TEXT, userid INT, first_name TEXT, last_name TEXT, PRIMARY KEY (song,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as file:
    csvreader = csv.reader(file)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO users_song (song, userid, first_name, last_name)"
        query = query + " VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

session.execute("DROP TABLE IF EXISTS song_length")
session.execute("DROP TABLE IF EXISTS song_playlist_session")
session.execute("DROP TABLE IF EXISTS users_song")

session.shutdown()
cluster.shutdown()